In [3]:
using Revise

using Test
import TensorCrossInterpolation as TCI
import Random

In [4]:
using Test
import TensorCrossInterpolation as TCI
import TensorCrossInterpolation: rank, linkdims, TensorCI2, MultiIndex, evaluate, crossinterpolate2, pivoterror, tensortrain
import Random
import QuanticsGrids as QD

In [5]:
import TensorCrossInterpolation as TCI
using TensorCrossInterpolation
using Random
using Test
using ITensors
ITensors.disable_warn_order()

14

In [7]:
pivotsearch = :full
seed = 124

Random.seed!(seed)

R = 30
abstol = 1e-5
grid = QD.DiscretizedGrid{1}(R, (0.0,), (1.0,))

f(bitlist) = fx(QD.quantics_to_origcoord(grid, bitlist)[1])

B = 2^(-30) # global variable

nc = 1000
coeffs = [randn() for _ in 1:nc]
exps = 10 .* rand(nc)

function fx(x)
    sum(c * exp(- e * x^2) for (c, e) in zip(coeffs, exps))
end

localdims = fill(2, R)
firstpivot = ones(Int, R)
tci, ranks, errors = crossinterpolate2(
    Float64,
    f,
    localdims,
    [firstpivot];
    tolerance=abstol,
    maxbonddim=1000,
    maxiter=20,
    loginterval=1,
    verbosity=1,
    normalizeerror=false,
    pivotsearch=pivotsearch,
)
@show abstol

iteration = 1, rank = 2, error= 7.560125703354181e-6, maxsamplevalue= 7.69618831338151, nglobalpivot=0
iteration = 2, rank = 5, error= 7.585593870789299e-6, maxsamplevalue= 7.69618831338151, nglobalpivot=0
iteration = 3, rank = 5, error= 9.765775984938753e-6, maxsamplevalue= 7.69618831338151, nglobalpivot=0
iteration = 4, rank = 5, error= 7.560125665939665e-6, maxsamplevalue= 7.69618831338151, nglobalpivot=0
abstol = 1.0e-5


1.0e-5

In [9]:
#for orthocenter in [1, length(tci), length(tci) ÷ 2]
for orthocenter in [1]
    tt = TCI.TensorTrain(TCI.sitetensors(tci, f))
    TCI.compress!(tt, :SVD; tolerance=1e-8)
    println("Orthocenter: ", orthocenter)
    @show TCI.linkdims(tt)
    println("Error on pivots:")
    for b in 1:length(tci)-1
        Iset = TCI.Iset(tci, b+1)
        Jset = TCI.Jset(tci, b)
        diff = maximum(abs, [TCI.evaluate(tt, vcat(i, j)) - f(vcat(i, j)) for i in Iset, j in Jset])
        println(b, "    ", diff, " ", diff > abstol)
    end
    println("Error on T:")
    for l in 1:length(tci)-1
        diff = [TCI.evaluate(tt, vcat(i, m, j)) - f(vcat(i, m, j)) for i in TCI.Iset(tci, l), j in TCI.Jset(tci, l), m in 1:localdims[l]]
        println(l, "    ", maximum(abs, diff), " ", maximum(abs, diff) > abstol)
    end

    println("Error on Π:")
    for b in 1:length(tci)-1
        Iset_b = TCI.Iset(tci, b)
        Jset_bp1 = TCI.Jset(tci, b+1)
        diff = [
            TCI.evaluate(tt, vcat(i, i1, i2, j)) - f(vcat(i, i1, i2, j)) for i in Iset_b, j in Jset_bp1, i1 in 1:localdims[b], i2 in 1:localdims[b+1]]
        println(b, "    ", maximum(abs, diff), " ", maximum(abs, diff) > abstol)
    end
end

Orthocenter: 1
TCI.linkdims(tt) = [2, 4, 5, 5, 4, 4, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Error on pivots:
1    4.5845932383770105e-7 false
2    8.187634650269615e-7 false
3    8.187634650269615e-7 false
4    1.2965773361628408e-6 false
5    1.0932180414258141e-6 false
6    6.638858307539408e-7 false
7    9.031265246761677e-8 false
8    2.5303198922799197e-6 false
9    1.6200759400675224e-6 false
10    6.191414794187722e-7 false
11    8.855831734599917e-8 false
12    8.858004818534937e-8 false
13    8.858809763534481e-8 false
14    8.859172950792527e-8 false
15    8.85935121930359e-8 false
16    8.859439371011746e-8 false
17    8.859480304934664e-8 false
18    1.7208456881689926e-14 false
19    1.7208456881689926e-14 false
20    1.7208456881689926e-14 false
21    4.440892098500626e-15 false
22    4.440892098500626e-15 false
23    4.440892098500626e-15 false
24    4.440892098500626e-15 false
25    4.440892098500626e-15 false
26    4.440892098500626e-15 fa

In [ ]:
tt = TCI.TensorTrain(TCI.sitetensors(tci, f))
tt_old = TCI.TensorTrain(TCI.sitetensors_site0update(tci, f))

@show TCI.linkdims(tt)
@show TCI.linkdims(tt_old)

In [ ]:
for l in 1:length(tci)
    Iset = TCI.Iset(tci, l)
    Jset = TCI.Jset(tci, l)
    println(l, "    ", maximum(abs, [TCI.evaluate(tt, vcat(i, l, j)) - f(vcat(i, l, j)) for i in Iset, j in Jset, l in 1:localdims[l]]))
end

In [ ]:
maximum([abs(TCI.evaluate(tt, p) - f(p)) for p in tci.globalpivots])

In [ ]:
for b in 1:length(tci)-1
    Iset = TCI.Iset(tci, b+1)
    Jset = TCI.Jset(tci, b)
    println(b, "    ", maximum(abs, [TCI.evaluate(tt_old, vcat(i, j)) - f(vcat(i, j)) for i in Iset, j in Jset]))
end

maximum([abs(TCI.evaluate(tt_old, p) - f(p)) for p in tci.globalpivots])

In [ ]:
using Plots

p = plot(ylims=(0, 2))
b = 5
plot!(p, sort(vec(abs.(tt[b]))))
plot!(p, sort(vec(abs.(tt_old[b]))))

In [ ]:
length(tci.globalpivots)